# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [12]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig, Model
import requests
import json

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# get the current workspace

ws = Workspace.from_config()

In [3]:
# create the dataset, if not exists

found = False
key = "Debts"
description_text = "Debts dataset from Brazilian Telefony Company"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://www.jlnsoftware.com.br/azure_ml/Debts.csv'
        dataset = TabularDatasetFactory.from_delimited_files(example_data,separator=";")        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
dataset

{
  "source": [
    "https://www.jlnsoftware.com.br/azure_ml/Debts.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "a586f540-3962-433d-89ce-9c0f53553fd0",
    "name": "Debts",
    "version": 1,
    "description": "Debts dataset from Brazilian Telefony Company",
    "workspace": "Workspace.create(name='quick-starts-ws-259246', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-259246')"
  }
}

In [4]:
# create the cluster, if not exists

amlcompute_cluster_name = "cluster-6-ds3-v2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority = 'dedicated',
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
# choose a name for experiment

experiment_name = 'Debts-Hyperdrive'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [32]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.

early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training

param_sampling = RandomParameterSampling({
        "--criterion": choice('gini', 'entropy'),
        "--splitter": choice('best', 'random'),
        "--max_depth": choice(0,10,100,200),
        "--min_samples_split": choice(2,5,10,20),
        "--min_samples_leaf": choice(1,5,10,20)
    })

#TODO: Create your estimator and hyperdrive config
# based on https://github.com/microsoft/MLHyperparameterTuning/blob/master/07_Train_With_AML_Pipeline.ipynb and https://learn.microsoft.com/pt-br/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py

estimator = Estimator(source_directory="./hd_source",
                      entry_script='train.py',
                      compute_target=compute_target,
                      conda_dependencies_file="hd_env.yml")


hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="AUC_weighted",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=5,
                             max_duration_minutes=60)

In [33]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [34]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
# make sure that the experiment is finished

hyperdrive_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [39]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n AUC_weighted:', best_run_metrics['AUC_weighted'])
print('\n criterion:',parameter_values[1])
print('\n max_depth:',parameter_values[3])
print('\n min_samples_split:',parameter_values[5])
print('\n min_samples_leaf',parameter_values[7])
print('\n splitter:',parameter_values[9])

best_run

Best Run Id:  HD_518b48d7-17b3-4b86-bfe4-28db6ce921d7_5

 AUC_weighted: 0.9873905650214171

 criterion: entropy

 max_depth: 0

 min_samples_split: 5

 min_samples_leaf 5

 splitter: best


Experiment,Id,Type,Status,Details Page,Docs Page
Debts-Hyperdrive,HD_518b48d7-17b3-4b86-bfe4-28db6ce921d7_5,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [36]:
# Save the best model in Azure

model = best_run.register_model(model_name="Debts-Hyperdrive-bestmodel", description="The best model for Debts dataset using Hyperdrive", model_path='outputs/model.pkl')

# Save the best model on disk

best_run.download_file(name='outputs/model.pkl',output_file_path='Debts-Hyperdrive-bestmodel.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Not the best model, AutoML outperform Hyperdrive best model

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Not the best model, AutoML outperform Hyperdrive best model

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Deletion of compute cluster

compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

